# ตัวอย่างการใช้งาน Gemini in Python อย่างง่าย

Notebook นี้สาธิตวิธีการใช้งาน Gemini API เพื่อ:
- สร้างข้อความจาก prompt
- วิเคราะห์ภาพ (Image Understanding)
- อ่านและสรุปไฟล์ PDF (PDF Understanding)
----

1. ติดตั้งไลบรารี `google-genai` ด้วยคำสั่ง pip
2. API Key สำหรับการใช้งาน Gemini API
3. Gemini Generate text
4. Gemini image understanging
5. Gemini pdf understanging


## 1. ติดตั้งไลบรารี `google-genai` ด้วยคำสั่ง pip

In [ ]:
# Installation
!pip install google-genai

## 2. API Key สำหรับการใช้งาน Gemini API
<img src="https://storage.googleapis.com/punsiriboo_public/ai-studio1.png" width="500">
<img src="https://storage.googleapis.com/punsiriboo_public/ai_studio2.png" width="500">

ก่อนจะเริ่มต้นใช้งาน Gemini API ผ่าน Python เราจำเป็นต้องมี **API Key** ซึ่งสามารถสร้างได้จาก Google AI Studio

วิธีการขอ API Key

1. เข้าเว็บไซต์ [Google AI Studio](https://aistudio.google.com/app/apikey)
2. **Sign in** ด้วยบัญชี Google ของคุณ
3. คลิกปุ่ม `Create API Key`
4. ตั้งชื่อ Key และเลือก Project ที่ต้องการ (หรือสร้าง Project ใหม่)
5. กด "Create" แล้ว **คัดลอก API Key** ไว้ใช้งาน

> 🔑 แนะนำให้เก็บ API Key ไว้ใน `.env` file หรือ Environment Variables เพื่อความปลอดภัย

In [ ]:
import os
from dotenv import load_dotenv
from google import genai

load_dotenv("../.env")

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

## 3. Gemini: Generate Text

Gemini API รองรับการสร้างข้อความจาก prompt ได้อย่างชาญฉลาด  
เหมาะกับการใช้ในงาน เช่น การเขียนบทความ แนะนำสินค้า สรุปข้อมูล หรือแม้แต่แต่งเรื่อง

In [ ]:
# กำหนดโมเดล Gemini ที่จะใช้ (เวอร์ชันล่าสุดและเร็ว)
MODEL_ID = "gemini-2.5-flash"

# เรียกใช้ Gemini เพื่อให้แนะนำหนังสือ AI สำหรับผู้เริ่มต้น
response = client.models.generate_content(
    model=MODEL_ID,  # ระบุโมเดลที่ใช้
    contents='แนะนำหนังสือ AI สำหรับผู้เริ่มต้น'  # ข้อความที่ใช้ถาม AI
)

# แสดงผลลัพธ์ที่ได้จากโมเดล
print(response.text)

## 4. Gemini Image Understanding

Google Gemini API รองรับการประมวลผลภาพร่วมกับข้อความ (multimodal)  
ทำให้คุณสามารถส่ง **รูปภาพ + คำถาม** เพื่อให้โมเดลวิเคราะห์และให้คำตอบได้ เช่น  
การตรวจสอบว่าสินค้าในภาพคืออะไร, วิเคราะห์ป้ายราคา, หรือเข้าใจบริบทของภาพในเชิงลึก

---
ตัวอย่างภาพ

<img src="https://storage.googleapis.com/ai-agent-jetpack/example-product.png" width="100px;"/>


In [ ]:
!wget https://storage.googleapis.com/ai-agent-jetpack/example-product.png

In [ ]:
import PIL.Image  # ใช้จากไลบรารี Pillow สำหรับเปิดไฟล์ภาพ

# โหลดภาพจากไฟล์ในเครื่อง
image = PIL.Image.open("example-product.png")

# เรียก Gemini API โดยใช้โมเดลที่ตั้งไว้ เช่น gemini-2.0-flash
response = client.models.generate_content(
    model=MODEL_ID,                  # ใช้โมเดล Gemini ที่กำหนดไว้ก่อนหน้า
    contents=[
        "ภาพนี้คืออะไร?",           # คำสั่ง/คำถามที่ส่งเข้าไป
        image,                       # ภาพที่ต้องการให้ AI วิเคราะห์
    ],
)

# แสดงผลลัพธ์ที่ AI ตอบกลับมา
print(response.text)

## 5. Gemini Document Understanding

Gemini รองรับการวิเคราะห์เอกสาร PDF โดยใช้โมเดลเวอร์ชัน `gemini-1.5-pro` หรือสูงกว่า  
สามารถใช้ในการสรุปเนื้อหา ค้นหาข้อมูลในเอกสาร หรือถามตอบจากไฟล์ PDF โดยตรง

---

ตัวอย่างเอกสาร PDF

<img src="https://storage.googleapis.com/ai-agent-jetpack/example-pdf.png" width="300px"></img>


In [ ]:
import httpx  # ใช้สำหรับส่ง HTTP request
from google.genai import types  # ใช้สร้างข้อมูลประเภท Part (เช่น PDF, image)

# ลิงก์ไปยัง PDF เอกสาร Fund Factsheet
doc_url = "https://secdocumentstorage.blob.core.windows.net/fundfactsheet/M0370_2563.pdf"

# โหลดเอกสาร PDF จาก URL เป็น binary
doc_data = httpx.get(doc_url).content

# คำถามที่ส่งไปให้ Gemini วิเคราะห์
prompt = "สรุปเอกสารนี้ให้หน่อย"

# เรียกใช้ Gemini API โดยแนบทั้ง PDF และ prompt
response = client.models.generate_content(
  model=MODEL_ID,   # เช่น gemini-2.0-flash
  contents=[
      types.Part.from_bytes(
        data=doc_data,               # ข้อมูล binary ของ PDF
        mime_type='application/pdf' # MIME type บอกว่าเป็น PDF
      ),
      prompt                         # สั่งให้สรุปเอกสาร
  ]
)

# แสดงผลลัพธ์จาก Gemini
print(response.text)